In [1]:
import os
import sys
sys.path.append('../src')

from openai import OpenAI
from dotenv import load_dotenv
from agents import BaseAgent, Agent, MultiAgent

In [2]:
load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_BASE_URL"),
)

In [3]:
# Top level agent
general_agent = Agent(base_agent = BaseAgent(client),
           name = "general agent",
           instructions= "Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
           handoff_description = "Call this agent if you don't know how to answer the user's question or do not has access to the necessary information.")

# Down level agents
personal_assistant_agent = Agent(base_agent = BaseAgent(client),
              name = "personal assistant agent",
              instructions= "Help manage schedules, send reminders for tasks, and provide information on daily activities. Assist with time management and organization.",
              handoff_description = "Call this agent if a user is asking about a topic related to personal assistance like scheduling, reminders, and time management.")

budgeting_and_finance_agent = Agent(base_agent = BaseAgent(client),
                name = "budgeting and finance agent",
                instructions= "Assist with managing personal finances, tracking expenses, and providing advice on saving and budgeting strategies to ensure financial well-being.",
                handoff_description = "Call this agent if a user is asking about a topic related to budgeting and finance like managing personal finances, tracking expenses, and saving strategies.")

In [4]:
# some tools for personal assistant agent
def get_schedule():
    """ 
    Returns a list of events on the schedule.
    """
    schedule = ["meeting with John at 10am", "lunch with Jane at 12pm"]
    return f"your schedule for today is: {', '.join(schedule)}"
def get_reminders():
    """ 
    Returns a list of reminders.
    """
    reminders = ["buy groceries", "pay bills"]
    return f"your reminders are: {', '.join(reminders)}"
# some tools for budgeting and finance agent
def get_today_total_cost():
    """ 
    Returns today's total cost.
    """
    cost = 100
    return f"today's total cost is {cost}"
def get_total_cost_this_month():
    """
    Returns this month's total cost.
    """
    cost = 3000
    return f"this month's total cost is {cost}"

personal_assistant_agent.add_tools([get_schedule, get_reminders])
budgeting_and_finance_agent.add_tools([get_today_total_cost, get_total_cost_this_month])

In [5]:
# MultiAgent

multi_agent = MultiAgent(start_agent=general_agent)
# top level agent hands off to down level agents
multi_agent.add_handoff_relations(from_agent=general_agent,to_agents=[personal_assistant_agent,budgeting_and_finance_agent])


In [10]:
multi_agent.chat([{ "role": "user", "content": "how much did I spend today?"}],agent=general_agent)
# multi_agent.handoff([{"role": "user", "content": "I need help with my budget."}],agent=general_agent)

[{'role': 'assistant',
  'content': "You spent a total of $100 today. If you need help tracking where that money went or if you're looking for budgeting advice, feel free to share!",
  'sender': 'budgeting_and_finance_agent'}]

In [7]:
multi_agent.chat([{ "role": "user", "content": "how much did I spend today?"}],agent=general_agent,show_details=True)

[{'role': 'assistant',
  'tool_calls': [{'id': 'call_1t54vKq2brKz4wkyPtF7E1xw',
    'function': {'arguments': '{"message": "how much did I spend today?"}',
     'name': 'budgeting_and_finance_agent'},
    'type': 'function'}],
  'type': 'handoff',
  'sender': 'general_agent'},
 {'role': 'tool',
  'handoff': 'general_agent -> budgeting_and_finance_agent',
  'content': 'Handing off to budgeting_and_finance_agent',
  'tool_call_id': 'call_1t54vKq2brKz4wkyPtF7E1xw',
  'message': 'how much did I spend today?',
  'type': 'handoff',
  'sender': 'budgeting_and_finance_agent'},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_2KbTP4SmXuc5hqRoEXGjvxzl',
    'function': {'arguments': '{}', 'name': 'get_today_total_cost'},
    'type': 'function'}],
  'type': 'tool',
  'sender': 'budgeting_and_finance_agent'},
 {'role': 'tool',
  'content': "today's total cost is 100",
  'tool_call_id': 'call_2KbTP4SmXuc5hqRoEXGjvxzl',
  'type': 'tool',
  'sender': 'budgeting_and_finance_agent'},
 {'role': 'ass

In [8]:
multi_agent.chat([{ "role": "user", "content": "what's on my schedule today?"}],agent=general_agent)

[{'role': 'assistant',
  'content': 'Your schedule for today includes:\n\n- **10:00 AM**: Meeting with John\n- **12:00 PM**: Lunch with Jane\n\nWould you like to set reminders for these events or need any additional information?',
  'sender': 'personal_assistant_agent'}]

In [9]:
multi_agent.chat([{ "role": "user", "content": "how much did I spend today and what's on my schedule?"}],agent=general_agent,show_details=False)

[{'role': 'assistant',
  'content': 'Today, your total spending amounted to $100. If you would like, I can help you break down this spending or provide insights on how to adjust your budget for future expenses.',
  'sender': 'budgeting_and_finance_agent'},
 {'role': 'assistant',
  'content': "Today's schedule includes:\n\n- **10:00 AM**: Meeting with John\n- **12:00 PM**: Lunch with Jane\n\nIf you need any reminders set or additional information, let me know!",
  'sender': 'personal_assistant_agent'}]